In [1]:
# Copyright (c) 2015 Matthew Earl
# 
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
# 
#     The above copyright notice and this permission notice shall be included
#     in all copies or substantial portions of the Software.
# 
#     THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS
#     OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF
#     MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN
#     NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM,
#     DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR
#     OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE
#     USE OR OTHER DEALINGS IN THE SOFTWARE.

In [1]:
import cv2
import dlib
import numpy
import sys
from time import sleep

PREDICTOR_PATH = "shape_predictor_68_face_landmarks.dat"
SCALE_FACTOR = 1
FEATHER_AMOUNT = 11

FACE_POINTS = list(range(17,68))
MOUTH_POINTS = list(range(48,61))
RIGHT_BROW_POINTS = list(range(17,22))
LEFT_BROW_POINTS = list(range(22,27))
RIGHT_EYE_POINTS = list(range(36,42))
LEFT_EYE_POINTS = list(range(42,48))
NOSE_POINTS = list(range(27,35))
JAW_POINTS = list(range(0,17))
ALIGN_POINTS = (LEFT_BROW_POINTS+RIGHT_EYE_POINTS+LEFT_EYE_POINTS+
                RIGHT_BROW_POINTS+NOSE_POINTS+MOUTH_POINTS)
OVERLAY_POINTS=[
    LEFT_EYE_POINTS+RIGHT_EYE_POINTS+LEFT_BROW_POINTS+RIGHT_BROW_POINTS,
    NOSE_POINTS+MOUTH_POINTS,
]
COLOR_CORRECT_BLUR_FRAC=0.6


cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(PREDICTOR_PATH)


def get_landmarks(img, dlibOn):
    if (dlibOn == True):
        rects = detector(img, 1)
        if len(rects) > 1:
            return "error"
        if len(rects) == 0:
            return "error"
        return numpy.matrix([[p.x, p.y] for p in predictor(img, rects[0]).parts()])

    else:
        rects = cascade.detectMultiScale(img, 1.3, 5)
        if len(rects) > 1:
            return "error"
        if len(rects) == 0:
            return "error"
        x, y, w, h = rects[0]
        rect = dlib.rectangle(x, y, x + w, y + h)
        return numpy.matrix([[p.x, p.y] for p in predictor(img, rect).parts()])


def annotate_landmarks(img, landmarks):
    img = img.copy()
    for idx, point in enumerate(landmarks):
        pos = (point[0, 0], point[0, 1])
        cv2.putText(img, str(idx), pos,
                    fontFace=cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,
                    fontScale=0.4,
                    color=(0, 0, 255))
        cv2.circle(img, pos, 3, color=(0, 255, 255))
    return img


def draw_convex_hull(img, points, color):
    points = cv2.convexHull(points)
    cv2.fillConvexPoly(img, points, color=color)


def get_face_mask(img, landmarks):
    img = numpy.zeros(img.shape[:2], dtype=numpy.float64)

    for group in OVERLAY_POINTS:
        draw_convex_hull(img,
                         landmarks[group],
                         color=1)

    img = numpy.array([img, img, img]).transpose((1, 2, 0))

    img = (cv2.GaussianBlur(img, (FEATHER_AMOUNT, FEATHER_AMOUNT), 0) > 0) * 1.0
    img = cv2.GaussianBlur(img, (FEATHER_AMOUNT, FEATHER_AMOUNT), 0)

    return img


def transformation_from_points(points1, points2):


    points1 = points1.astype(numpy.float64)
    points2 = points2.astype(numpy.float64)

    c1 = numpy.mean(points1, axis=0)
    c2 = numpy.mean(points2, axis=0)
    points1 -= c1
    points2 -= c2

    s1 = numpy.std(points1)
    s2 = numpy.std(points2)
    points1 /= s1
    points2 /= s2

    U, S, Vt = numpy.linalg.svd(points1.T * points2)

    R = (U * Vt).T

    return numpy.vstack([numpy.hstack(((s2 / s1) * R,
                                       c2.T - (s2 / s1) * R * c1.T)),
                         numpy.matrix([0., 0., 1.])])


def read_img_and_landmarks(fname):
    img = cv2.imread(fname, cv2.IMREAD_COLOR)
    img = cv2.resize(img, None, fx=0.35, fy=0.35, interpolation=cv2.INTER_LINEAR)
    img = cv2.resize(img, (img.shape[1] * SCALE_FACTOR,
                         img.shape[0] * SCALE_FACTOR))
    s = get_landmarks(img, dlibOn)

    return img, s


def warp_img(img, M, dshape):
    output_img = numpy.zeros(dshape, dtype=img.dtype)
    cv2.warpAffine(img,
                   M[:2],
                   (dshape[1], dshape[0]),
                   dst=output_img,
                   borderMode=cv2.BORDER_TRANSPARENT,
                   flags=cv2.WARP_INVERSE_MAP)
    return output_img


def correct_colors(img1, img2, landmarks1):
    blur_amount = COLOR_CORRECT_BLUR_FRAC * numpy.linalg.norm(
        numpy.mean(landmarks1[LEFT_EYE_POINTS], axis=0) -
        numpy.mean(landmarks1[RIGHT_EYE_POINTS], axis=0))
    blur_amount = int(blur_amount)
    if blur_amount % 2 == 0:
        blur_amount += 1
    img1_blur = cv2.GaussianBlur(img1, (blur_amount, blur_amount), 0)
    img2_blur = cv2.GaussianBlur(img2, (blur_amount, blur_amount), 0)

    # Avoid divide-by-zero errors.
    img2_blur += (128 * (img2_blur <= 1.0)).astype(img2_blur.dtype)

    return (img2.astype(numpy.float64) * img1_blur.astype(numpy.float64) /
            img2_blur.astype(numpy.float64))


# main function which is used in swaping frame and  filter image

def face_swap(img,name):

    # get landmarks with dblib true
    s = get_landmarks(img,True)
    if ( s=="error" ):
        print("No or too many faces")
        return img

    # getting image and landmarks of both the images
    img1,landmarks1 = img,s
    img2,landmarks2 = read_img_and_landmarks(name)

    # finding the orthogonal matrix by combining both the matrix
    M = transformation_from_points(landmarks1[ALIGN_POINTS],
                                   landmarks2[ALIGN_POINTS])

    # getting mask for filter image
    mask = get_face_mask(img2,landmarks2)
    # warping the filter image
    warped_mask = warp_img(mask,M,img1.shape)

    # combining both (image1,landmark1) +(image2 ,landmark2)
    combined_mask = numpy.max([get_face_mask(img1,landmarks1),warped_mask],axis=0)

    warped_img2 = warp_img(img2,M,img1.shape)

    # correcting with landmarks1 again
    warped_corrected_img2 = correct_colors(img1,warped_img2,landmarks1)

    output_img = img1 * (1.0-combined_mask)+warped_corrected_img2*combined_mask

    cv2.imwrite('output.jpg',output_img)
    image = cv2.imread('output.jpg')

    frame = cv2.resize(image,None,fx=1.5,fy=1.5,interpolation=cv2.INTER_LINEAR)

    return  image



capture = cv2.VideoCapture(-1)

filterImage = "kim.jpg"
dlibOn = False

while True:
    ret,frame = capture.read()
    # resizing is done to reduce the pixel intensity
    frame = cv2.resize(frame,None,fx=0.75,fy=0.75,interpolation=cv2.INTER_LINEAR)
    # image is flipper for mirroring purpose
    frame = cv2.flip(frame,1)

    #frame and filter image are fed to face_swap function
    cv2.imshow("Face swapper",face_swap(frame,filterImage))
    if cv2.waitKey(1) == 13:
        break

capture.release()
cv2.destroyAllWindows()

/home/robot/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:155: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too many faces
No or too man